## Record and Save as Json

In [1]:
import pandas as pd
from pose import img_keypoints_json
from pose import video_keypoints_json
from pose import img_keypoints_csv
from pose import video_keypoints_csv
from pose import play_csv
from pose import play_json
from pose import display_images
from pose import display_keypoints_imgs
from pose import cam2csv
from pose import cam2json

In [ ]:
cam2csv('you', 'you.csv')

In [ ]:
cam2json('you', 'you.json')

## Display Images

Accepts a list of image paths as the only argument, if you wanna check out a single picture than make a list with a single element which is that picture. 

example

img = ['img.jpg']

In [ ]:
import os
import glob

# Path to the 'pushups' folder
pushups_folder = 'data/pushups/images'

# Get all image paths (matching jpg, jpeg, png)
image_paths = glob.glob(os.path.join(pushups_folder, '*.[jJpP][pPnN][gG]')) + \
              glob.glob(os.path.join(pushups_folder, '*.jpeg'))

# Print or use the image paths
print(image_paths)

In [2]:
img = ['doug.jpg']

In [3]:
#press any key to continue
display_images(img)

## Convert Pictures to CSV and JSON

In [ ]:
"""
takes a list of pictures, a label, and a filename for the output CSV file as input
"""
img_keypoints_csv(img, 'doug', 'doug.csv')
img_keypoints_json(img, 'pushups', 'doug.json')

In [3]:
df = pd.read_csv('doug.csv')
df

,label,x_0,y_0,z_0,visibility_0,x_1,y_1,z_1,visibility_1,x_2,...,z_30,visibility_30,x_31,y_31,z_31,visibility_31,x_32,y_32,z_32,visibility_32
0,doug,0.516238,0.289408,-0.311132,0.999975,0.523862,0.278599,-0.283088,0.999908,0.529568,...,0.43088,0.884,0.542015,0.789269,-0.014898,0.982699,0.456543,0.765679,0.269362,0.975242


### Visualize Keypoints in the data frame

In [4]:
#press any key to scroll through pictures
display_keypoints_imgs(df)

## Convert Key Points in Video To CSV

takes three arguments
1. a list of videos
2. a label for the video, which will be added as a column in the csv
3. a name for it to be saved as

the below example takes a video i found on youtube and converts it to a csv

In [7]:
vid = ['data/vids/s03/58860488/standing_ab_twists.mp4']
video_keypoints_csv(vid, 's03_standing_ab_twists', 's03_standing_ab_twists_br.csv')


Processing video: data/vids/s03/60457274/standing_ab_twists.mp4
Keypoints from 1 videos labeled with 's03_standing_ab_twists' saved to s03_standing_ab_twists_fl.csv.


In [ ]:
df = pd.read_csv('skating.csv')
df

## replay 

play_csv takes the csv file produced from video_keypoints_csv() and replays the video of just the skeleton and keypoints against a black frame 

In [6]:
play_csv('skating.csv')

## Convert Key Points in Video To JSON

we can basically do the same thing with json files

In [ ]:
vid = ['data/dance/video/dance.mp4']
video_keypoints_json(vid, 'dance', 'dance.json')

### Replay Json

In [8]:
play_json('dance.json')